<a href="https://colab.research.google.com/github/saifulislamsarfaraz/Machine-Learning/blob/main/Llama_3_8b_Bangla_Inference_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Dependencies

**Run this cell otherwise none of the cell will work**

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

# FastLanguageModel from unsloth for 2x faster inference

**Loading the model**

In [ ]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "KillerShoaib/llama-3-8b-bangla-lora", # or  KillerShoaib/llama-3-8b-bangla-4bit
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


**Inferencing**

In [ ]:
# alpaca_prompt for the model
alpaca_prompt = """Below is an instruction in bangla that describes a task, paired with an input also in bangla that provides further context. Write a response in bangla that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# input with instruction and input
inputs = tokenizer(
[
    alpaca_prompt.format(
        "সুস্থ থাকার তিনটি উপায় বলুন", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# generating the output and decoding it
outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction in bangla that describes a task, paired with an input also in bangla that provides further context. Write a response in bangla that appropriately completes the request.\n\n### Instruction:\nসুস্থ থাকার তিনটি উপায় বলুন\n\n### Input:\n\n\n### Response:\n১. ভারসাম্যপূর্ণ খাদ্য গ্রহণ করুন: একটি সুষম খাদ্য গ্রহণ করা আপনাকে প্রয়োজনীয় পুষ্টি সরবরাহ করতে সহায়তা করতে পারে যা আপনাকে শক্তি এবং সুস্থতা বজায় রাখতে পারে। এর মধ্যে প্রচুর পরিমাণে ফল, সবজি, পূর্ণ শস্য, পাতলা প্রোটিন এবং স্বাস্থ্যকর ফ্যাট অন্তর্ভুক্ত রয়েছে। ২. নিয়মিত ব্যায়াম করুনঃ নিয়মিত শারীরিক ক্রিয়াকলাপ আপনার সামগ্রিক স্বাস্থ্যের জন্য অপরিহার্য। এটি হৃদরোগের স্বাস্থ্য উন্নত করতে, পেশী শক্তি বাড়াতে, ওজন হ্রাস করতে এবং মানসিক স্বাস্থ্যের উন্নতি করতে সহায়তা করতে পারে। সপ্তাহে কমপক্ষে ১৫০ মিনিটের মাঝারি তীব্রতার শারীরিক ক্রিয়াকলাপে অংশ নেওয়া ভাল। ৩. পর্যাপ্ত ঘুম পানঃ ঘুম শারীরিক এবং মানসিক স্বাস্থ্যের জন্য অপরিহার্য। প্রতি রাতে কমপক্ষে ৭-৮ ঘন্টা ঘুমের লক্ষ্য রাখুন। এটি আপনার মেজাজ 

# `AutoModelForPeftCausalLM` from Hugginface (only for LoRA adapter model)

Loading the **LoRA** model

In [ ]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model = AutoPeftModelForCausalLM.from_pretrained(
    "KillerShoaib/llama-3-8b-bangla-lora",
    load_in_4bit = True,
)
tokenizer = AutoTokenizer.from_pretrained("KillerShoaib/llama-3-8b-bangla-lora")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:159: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now set to True since model is quantized.


...

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


**Inferencing**

In [ ]:
alpaca_prompt = """Below is an instruction in bangla that describes a task, paired with an input also in bangla that provides further context. Write a response in bangla that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "নিচের আলোচনা দেখে বাংলাদেশের রাজধানীর নাম বের করো।", # instruction
        "বাংলাদেশের রাজধানী ঢাকায় তার নামে একটি এলাকার নামকরন করা হয়েছে শেরেবাংলা নগর (পূর্ববর্তী আইয়ুবনগর ও তারও পূর্বে মনিপুর) ,২০১৪ বাংলাদেশ স্থানীয় সময় শুক্রবার বেলা তিনটায় রাজধানীর ইউনাইটেড হাসপাতালে চিকিৎসাধীন অবস্থায় মারা যান বেবী মওদুদ। মৃত্যুর সময় তার বয়স হয়েছিল ৬৬ বছর।বাংলাদেশ জাতীয় জাদুঘর বাংলাদেশের রাজধানী ঢাকা শহরে অবস্থিত দেশের প্রধান জাদুঘর। এটি ২০, মার্চ, ১৯১৩ খ্রিস্টাব্দে প্রতিষ্ঠিত হয়, এবং ৭ আগস্ট।", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction in bangla that describes a task, paired with an input also in bangla that provides further context. Write a response in bangla that appropriately completes the request.\n\n### Instruction:\nনিচের আলোচনা দেখে বাংলাদেশের রাজধানীর নাম বের করো।\n\n### Input:\nবাংলাদেশের রাজধানী ঢাকায় তার নামে একটি এলাকার নামকরন করা হয়েছে শেরেবাংলা নগর (পূর্ববর্তী আইয়ুবনগর ও তারও পূর্বে মনিপুর),২০১৪ বাংলাদেশ স্থানীয় সময় শুক্রবার বেলা তিনটায় রাজধানীর ইউনাইটেড হাসপাতালে চিকিৎসাধীন অবস্থায় মারা যান বেবী মওদুদ। মৃত্যুর সময় তার বয়স হয়েছিল ৬৬ বছর।বাংলাদেশ জাতীয় জাদুঘর বাংলাদেশের রাজধানী ঢাকা শহরে অবস্থিত দেশের প্রধান জাদুঘর। এটি ২০, মার্চ, ১৯১৩ খ্রিস্টাব্দে প্রতিষ্ঠিত হয়, এবং ৭ আগস্ট।\n\n### Response:\nবাংলাদেশের রাজধানী ঢাকা।<|end_of_text|>']

# `AutoModelForCausalLM` from Hugginface (llama-3-8b-bangla-4bit)

**Loading the model**

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "KillerShoaib/llama-3-8b-bangla-4bit"
tokenizer_name = model_name

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
# Load model
model = AutoModelForCausalLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

**Inferencing**

In [ ]:
# Text prompt to start generation
alpaca_prompt = """Below is an instruction in bangla that describes a task, paired with an input also in bangla that provides further context. Write a response in bangla that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Encode the prompt text
inputs = tokenizer(
[
    alpaca_prompt.format(
        "বিশ্বের সবচেয়ে বিখ্যাত চিত্রশিল্পী কে?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# output
outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)

tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction in bangla that describes a task, paired with an input also in bangla that provides further context. Write a response in bangla that appropriately completes the request.\n\n### Instruction:\nবিশ্বের সবচেয়ে বিখ্যাত চিত্রশিল্পী কে?\n\n### Input:\n\n\n### Response:\nপ্রতিটি দেশের নিজস্ব বিখ্যাত চিত্রশিল্পী রয়েছে, কিন্তু বিশ্বব্যাপী সর্বাধিক বিখ্যাত চিত্রশিল্পীরা হলেন লিওনার্দো দা ভিনসি, রেমব্র্যান্ট ভ্যান রাইন, সার জোহান ভ্যান রাইসব্রাইক, জ্যাকসন পল ফর্বস, এডওয়ার্ড হপার, পল সিজান, জোন মিশেল, সার পল সেজ্যান, এবং অন্যান্যদের মধ্যে। এই চিত্রশিল্পীদের মধ্যে বেশিরভাগই ইউরোপীয় ছিলেন, তবে জন সিংলার মতো অনেক বিখ্যাত চিত্রশিল্পী আমেরিকার ছিলেন।<|end_of_text|>']